In [25]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from glob import glob

In [9]:
files = glob('/app/data/*/*.csv')
files

['/app/data/2023-07/2023-07-12 06:00:00.000000 - 2023-07-13 23:59:59.999999.csv',
 '/app/data/2023-07/2023-07-08 18:00:00.000000 - 2023-07-10 11:59:59.999999.csv',
 '/app/data/2023-07/2023-07-07 00:00:00.000000 - 2023-07-08 17:59:59.999999.csv',
 '/app/data/2023-07/2023-07-10 12:00:00.000000 - 2023-07-12 05:59:59.999999.csv',
 '/app/data/2024-07/2024-07-12 06:00:00 - 2024-07-13 23:59:59.999999.csv',
 '/app/data/2024-07/2024-07-10 12:00:00 - 2024-07-12 05:59:59.999999.csv',
 '/app/data/2024-07/2024-07-07 00:00:00 - 2024-07-08 17:59:59.999999.csv',
 '/app/data/2024-07/2024-07-08 18:00:00 - 2024-07-10 11:59:59.999999.csv',
 '/app/data/2022-07/2022-07-07 00:00:00 - 2022-07-08 17:59:59.999999.csv',
 '/app/data/2022-07/2022-07-08 18:00:00 - 2022-07-10 11:59:59.999999.csv',
 '/app/data/2022-07/2022-07-12 06:00:00 - 2022-07-13 23:59:59.999999.csv',
 '/app/data/2022-07/2022-07-10 12:00:00 - 2022-07-12 05:59:59.999999.csv',
 '/app/data/2021-07/2021-07-08 18:00:00 - 2021-07-10 11:59:59.999999.csv

In [4]:
spark = SparkSession.builder\
    .appName("TestJob")\
    .master("spark://spark-master:7077")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/05 13:01:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
df = spark.read.csv('/app/data/2024-07/1.csv', header=True)

In [23]:
df.dtypes

[('record_type', 'string'),
 ('type', 'string'),
 ('time', 'string'),
 ('project', 'string'),
 ('collector', 'string'),
 ('router', 'string'),
 ('router_ip', 'string'),
 ('peer_asn', 'string'),
 ('peer_address', 'string'),
 ('prefix', 'string'),
 ('next_hop', 'string'),
 ('as_path', 'string'),
 ('communities', 'string')]

In [21]:
df.show(5)

+-----------+----+-----------------+----------+----------------+------+---------+--------+--------------------+-------------------+--------------------+--------------------+--------------------+
|record_type|type|             time|   project|       collector|router|router_ip|peer_asn|        peer_address|             prefix|            next_hop|             as_path|         communities|
+-----------+----+-----------------+----------+----------------+------+---------+--------+--------------------+-------------------+--------------------+--------------------+--------------------+
|     update|   W|1720310400.067833|routeviews|route-views.eqix|  NULL|     NULL|   16552|2001:504:0:2:0:1:...|2a06:de05:626b::/48|                NULL|                NULL|                NULL|
|     update|   W|1720310400.067833|routeviews|route-views.eqix|  NULL|     NULL|   16552|2001:504:0:2:0:1:...|2a06:de05:62c5::/48|                NULL|                NULL|                NULL|
|     update|   W|1720310

In [30]:
df_agg = df.groupby('peer_asn').agg(f.count('time')).collect()
df_agg

In [34]:
df_agg = df.groupby('prefix').agg(f.count('time')).collect()
df_agg

[Row(prefix='2604:dbc0::/32', count(time)=5973),
 Row(prefix='201.132.132.0/23', count(time)=69),
 Row(prefix='189.39.80.0/24', count(time)=368),
 Row(prefix='45.43.200.0/23', count(time)=65),
 Row(prefix='2607:f740:e04c::/48', count(time)=102),
 Row(prefix='24.155.233.0/24', count(time)=184),
 Row(prefix='2402:e580:74b6::/48', count(time)=13480),
 Row(prefix='192.119.174.0/24', count(time)=54),
 Row(prefix='184.61.102.0/24', count(time)=55),
 Row(prefix='111.213.192.0/18', count(time)=322),
 Row(prefix='2607:2c0:1::/48', count(time)=8909),
 Row(prefix='103.10.169.0/24', count(time)=3531),
 Row(prefix='112.109.17.0/24', count(time)=3536),
 Row(prefix='139.5.154.0/24', count(time)=3538),
 Row(prefix='196.61.21.0/24', count(time)=81),
 Row(prefix='41.59.119.0/24', count(time)=82),
 Row(prefix='14.254.208.0/24', count(time)=96),
 Row(prefix='14.241.16.0/21', count(time)=96),
 Row(prefix='14.180.28.0/22', count(time)=100),
 Row(prefix='14.178.112.0/20', count(time)=100),
 Row(prefix='14.16

In [35]:
len(df_agg)

1221929

In [6]:
# spark.stop()